## #3

In [1]:
import numpy as np
import pandas as pd 

import pandas_datareader.data as web

# visualization
import matplotlib.pyplot as plt

# do not show Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [2]:
m2 = web.DataReader(name='M2', data_source='fred', start='1980-1', end='2023-12' )
m2

,M2
DATE,
1980-11-03,1591.4
1980-11-10,1592.9
1980-11-17,1596.3
1980-11-24,1597.2
1980-12-01,1596.1
...,...
2021-01-04,19038.9
2021-01-11,19408.0
2021-01-18,19420.1


In [3]:
from statsmodels.tsa.api import adfuller, kpss

In [4]:
adf_stat, pval, usedlag, nobs, critical_values, BIC = adfuller(m2, regression='ct', autolag='BIC')
# тестовая статистика, её p-значение и критические значения
pval

1.0

The $p$-value $>0.05$ $\Rightarrow$ the time series is not stationary

In [5]:
kpss_stat, p_value, lags, crit = kpss(m2, regression='ct')
# тестовая статистика, её p-значение и критические значения
p_value

/var/folders/86/0g7053ts24sg4v1rcqmpph680000gn/T/ipykernel_58030/3784901810.py:1: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, p_value, lags, crit = kpss(m2, regression='ct')


0.01

The $p$-value is $<0.05$ $\Rightarrow$ the time series is not stationary

In [6]:
dm2=m2.diff().dropna()

In [7]:
adf_stat, pval, usedlag, nobs, critical_values, BIC = adfuller(dm2, regression='ct', autolag='BIC')
# тестовая статистика, её p-значение и критические значения
pval

1.7954386356829563e-13

The $p$-value $<0.05$ $\Rightarrow$ $\Delta m2$ is stationary

$d=1$

In [8]:
kpss_stat, p_value, lags, crit = kpss(dm2, regression='ct')
# тестовая статистика, её p-значение и критические значения
p_value

/var/folders/86/0g7053ts24sg4v1rcqmpph680000gn/T/ipykernel_58030/2987853739.py:1: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_stat, p_value, lags, crit = kpss(dm2, regression='ct')


0.01

The $p$-value $<0.05$ $\Rightarrow$ $\Delta m2$ is not stationary

In [9]:
ddm2=m2.diff().diff().dropna()
kpss_stat, p_value, lags, crit = kpss(ddm2, regression='ct')
# тестовая статистика, её p-значение и критические значения
p_value

/var/folders/86/0g7053ts24sg4v1rcqmpph680000gn/T/ipykernel_58030/1656306650.py:2: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, crit = kpss(ddm2, regression='ct')


0.1

The $p$-values $>0.05$ $\Rightarrow$ $\Delta^2 m2$ is stationary

The conclusion is the following:
    according to the KPSS test $d=2$
    according to the Dickey-Fuller test $d=1$

Let us estimate $arima(2,1,1)$ and $arima(1,2,1)$

In [10]:
from statsmodels.tsa.api import ARIMA
from statsmodels.stats.api import het_arch, acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_predict

In [11]:
mod1 = ARIMA(m2, order=(2,1,1), trend='n', missing='drop')
mod2 = ARIMA(m2, order=(1,2,1), trend='n', missing='drop')

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-

In [12]:
res1 = mod1.fit()
res2 = mod2.fit()

Let's test the residues for serial correlations

In [13]:
model_df = mod1.k_params-1
# to test we drop first d residuals (d=mod.k_diff)
acorr_ljungbox(res1.resid[mod1.k_diff:] , lags=[7], model_df=model_df)

,lb_stat,lb_pvalue
7,272.312679,1.012206e-57


In [14]:
model_df = mod2.k_params-1
# to test we drop first d residuals (d=mod.k_diff)
acorr_ljungbox(res2.resid[mod2.k_diff:] , lags=[7], model_df=model_df)

,lb_stat,lb_pvalue
7,278.719827,3.749689e-58


For both models the $p$-value is $<0.05$ $\Rightarrow$ they do not pass the test for serial correlation

Let us compare these two models using the cross-validation

In [15]:
from sktime.split import temporal_train_test_split, ExpandingWindowSplitter, SlidingWindowSplitter, SingleWindowSplitter
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError # Метрики MSE, MAE, MAPE

In [16]:
from sktime.forecasting.arima import ARIMA, AutoARIMA

In [17]:
forecaster1 = ARIMA(order=(2,1,1), trend='n')
forecaster2 = ARIMA(order=(1,2,1), trend='n')

In [19]:
# разбиваем выбору на обучающую (первые 80%) и тестовую
y_train, y_test = temporal_train_test_split(m2, train_size=0.8)

# Обучаем модель на тестовом множестве
forecaster1.fit(y_train)
forecaster2.fit(y_train)

# прогнозируем для тестовых наблюдений
y_pred1 = forecaster1.predict(fh = y_test.index)
y_pred2 = forecaster2.predict(fh = y_test.index)

# инициализируем метрику
metric = MeanSquaredError(square_root=False)

# вычисляем метрику на данных
print('cv1=', metric.evaluate(y_test, y_pred1),'cv2=', metric.evaluate(y_test, y_pred2))

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)


cv1= 698219.5527398641 cv2= 651282.671083597


$cv2$ < $cv1$ $\Rightarrow$ the second model is better

Automatic order selection

In [26]:
forecaster = AutoARIMA(information_criterion='aic', test='adf', seasonal=False)
forecaster.fit(m2)
forecaster.get_fitted_params() #['order']

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-

{'intercept': 4.576147686288079,
 'ar.L1': -0.49770807975864695,
 'ar.L2': 0.21725029726582437,
 'ar.L3': 0.011966978609151371,
 'ar.L4': 0.3700556161323696,
 'ar.L5': 0.3683350668936859,
 'ma.L1': 0.8770958674756917,
 'sigma2': 457.3034326308302,
 'order': (5, 1, 1),
 'seasonal_order': (0, 0, 0, 0),
 'aic': 18839.90817351739,
 'aicc': 18839.977040088408,
 'bic': 18885.10571450708,
 'hqic': 18856.462820991375}

In [27]:
acorr_ljungbox(forecaster.predict_residuals(m2).dropna() , lags=[7], model_df=5+1)

,lb_stat,lb_pvalue
7,9.133846,0.002509


The lb statistics of low $\Rightarrow$ no serial correlation

Let us test the presence of heteroskedasticity

In [29]:
lm_stat, lm_pval, f_stat, f_pval = het_arch(forecaster.predict_residuals(m2).dropna(), nlags=7, ddof=5+1)

lm_pval

5.016076729298625e-76

We reject $H_0$ $\Rightarrow$ there is heteroskedasticity in residues